# AutoGen Tool Use Example 

## Import the Needed Packages 

In [1]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

# Opik imports
import opik
from opik import track
from opik.evaluation.metrics import Hallucination, AnswerRelevance

In [2]:
# Set Opik environment variables
os.environ["OPIK_API_KEY"] = "Opik Key"
os.environ["OPIK_PROJECT_NAME"] = "react_agent"
os.environ["OPIK_WORKSPACE"] = "alvi77"
os.environ["OPIK_ENABLED"] = "true"

# Initialize Opik client
opik_client = opik.Opik()

## Creating the Client 

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `model` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

As a quick test, we will just run a simple prompt - `What is the capital of France`. 

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


## Defining the Functions 

In this example, we will give the agent access to a tool that is a function with a list of available vacation destinations and their availability. 

You can think that this would be a scenario where a travel agent might have an access to a travel database for example. 

As you go through this sample, feel free to try to define new functions and tools that the agent can call. 

In [4]:
from typing import Dict, List, Optional


@track(name="vacation_destinations_tool")
def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        result = city, destinations[city]
    else:
        result = city, "City not found"
    
       
    return result

@track(name="weather_check_tool")
def check_weather(city: str) -> tuple[str, str, str]:
    """
    Checks the current weather for a specific city
    
    Args:
        city (str): Name of the city to check weather for
        
    Returns:
        tuple: Contains city name, weather condition, and temperature
    """
    # Simple mock weather data
    weather_data = {
        "Barcelona": ("Sunny", "24°C"),
        "Tokyo": ("Cloudy", "18°C"),
        "Cape Town": ("Windy", "22°C"),
        "Vancouver": ("Rainy", "15°C"),
        "Dubai": ("Hot", "35°C"),
        "Berlin": ("Cool", "12°C"),
        "Paris": ("Mild", "19°C"),
        "London": ("Foggy", "16°C"),
    }

    if city in weather_data:
        condition, temp = weather_data[city]
        result = city, condition, temp
    else:
        result = city, "Unknown", "N/A"
    
    
    return result
# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Defining the Function Tool 
To have the agent use the `vacation_destinations` as a `FunctionTool`, we need to define it as one. 

We will also provide a description of the to tool which is helpful for the agent to identify what that tool is used for in relation to the task the user has requested. 

In [5]:
# Create both tools
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

get_weather = FunctionTool(
    check_weather, description="Check current weather conditions and temperature for any city."
)

## Defining the agent 

Now we can create the agent in the below code. We define the `system_message` to give the agent instructions on how to help users find vacation destinations. 

We also set the `reflect_on_tool_use` parameter to true. This allows use the LLM to take the response of the tool call and send the response using natural language. 

You can set the the parameter to false to see the difference. 

In [6]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations, get_weather],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Running the Agent 

Now we can run the agent with the initial user message asking to take a trip to Tokyo. 

You can change this city desintation to see how the agent responds to the availablity of the city. 

In [8]:
@track(name="travel_agent_conversation")
async def assistant_run(user_query: str = "I would like to take a trip to Tokyo") -> None:
    """
    Run the travel agent with Opik tracking for evaluation
    """
    
    # Track the full conversation
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )
    
    # Extract tool usage information for evaluation
    tool_calls = []
    for msg in response.inner_messages:
        if hasattr(msg, 'content') and isinstance(msg.content, list):
            for item in msg.content:
                if hasattr(item, 'name') and hasattr(item, 'arguments'):
                    tool_calls.append({
                        "tool_name": item.name,
                        "arguments": item.arguments,
                        "call_id": getattr(item, 'id', 'unknown')
                    })
    
    print(response.inner_messages)
    print(response.chat_message)
    
    return response


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=18), content=[FunctionCall(id='call_plnQZUNA4Bf8v9DuCuYYyPgt', arguments='{"city":"Tokyo"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content="('Tokyo', 'Unavailable')", call_id='call_plnQZUNA4Bf8v9DuCuYYyPgt')], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=119, completion_tokens=79) content="It looks like I'm unable to provide specific details about Tokyo at this moment. However, I can suggest that the best time to visit Tokyo typically falls during the spring (March to May) and autumn (September to November) when the weather is mild and pleasant, with cherry blossoms in spring and beautiful autumn foliage. If you have other destinations in mind, I can help with those as well!" type='TextMessage'


Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=119, completion_tokens=79), content="It looks like I'm unable to provide specific details about Tokyo at this moment. However, I can suggest that the best time to visit Tokyo typically falls during the spring (March to May) and autumn (September to November) when the weather is mild and pleasant, with cherry blossoms in spring and beautiful autumn foliage. If you have other destinations in mind, I can help with those as well!", type='TextMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=18), content=[FunctionCall(id='call_plnQZUNA4Bf8v9DuCuYYyPgt', arguments='{"city":"Tokyo"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content="('Tokyo', 'Unavailable')", call_id='call_plnQZUNA4Bf8v9DuCuYYyPgt')], type